# Plotting Functions

## Plot horizontal structure

In [17]:
def plot_horizontal_structure(
    frame_to_plot,
    zonal_gridpoints,
    meridional_gridpoints,
    time_points,
    zonal_velocity, 
    meridional_velocity, 
    column_temperature, 
    column_moisture, 
    specified_output_file_directory,
    specified_initial_condition_name,
    physical_parameters,
    simulation_parameters,
    plotting_parameters,
    
):

    import time
    import numpy as np
    import matplotlib.pyplot as plt
    plt.rcParams.update({'font.size':22})
    import matplotlib.ticker as mticker
    import matplotlib.colors as mcolors
    import matplotlib.gridspec as gs

    from tqdm import tqdm
    
    # Auxiliary Functions
    import sys
    sys.path.insert(0, '/home/disk/eos7/sressel/research/thesis-work/python/auxiliary_functions/')
    import ipynb.fs.full.mjo_mean_state_diagnostics as mjo
    from ipynb.fs.full.rounding_functions import round_out, round_to_multiple
    from ipynb.fs.full.modified_colormap import Ahmed21_colormap
    Ahmed_cmap = Ahmed21_colormap()
    from ipynb.fs.full.normalize_data import normalize_data
    
    if frame_to_plot > len(time_points):
        raise ValueError(f"Frame out of range of array size: {frame_to_plot} > {len(time_points)}")

    [
        SPECIFIC_HEAT, 
        LATENT_HEAT,
        WATER_DENSITY,
        COLUMN_AVERAGE_MASS,
        EARTH_RADIUS,
        METERS_PER_DEGREE,
        SECONDS_PER_DAY,
    ] = physical_parameters

    [
        simulation_moisture,
        fringe_region
    ] = simulation_parameters
    
    [
        xlims, ylims,
        zonal_quiver_plot_spacing, meridional_quiver_plot_spacing,
        save_plot, plotting_units,
        moisture_anomaly_scaling,
        grid_scaling
    ] = plotting_parameters

    # Set plotting options
    plt.style.use('default')
    plt.rcParams.update({'font.size':22})

    # Specify whether to plot the values in simulated units or converted units
    if plotting_units == 'natural':
        plotting_temperature = column_temperature
        temperature_units = r"$\frac{K \, kg}{m^2}$"
        plotting_moisture = column_moisture
        moisture_units = r"$\frac{K \, kg}{m^2}$"

    elif plotting_units == 'converted':
        # Convert the units of temperature and moisture
        average_temperature = np.copy(column_temperature)/COLUMN_AVERAGE_MASS
        equivalent_moisture = np.copy(column_moisture)/LATENT_HEAT*SPECIFIC_HEAT*1000/WATER_DENSITY

        plotting_temperature = average_temperature
        temperature_units = r"K"
        plotting_moisture = equivalent_moisture
        moisture_units = r"mm"

    if (not moisture_anomaly_scaling == 1) and (plotting_units == 'converted'):
        print(f"Fields scaled! Maximum moisture anomaly set to {moisture_anomaly_scaling:0.2f} mm")
        zonal_velocity *=  (moisture_anomaly_scaling/np.nanmax(np.abs(plotting_moisture[frame_to_plot])))
        meridional_velocity *=  (moisture_anomaly_scaling/np.nanmax(np.abs(plotting_moisture[frame_to_plot])))
        plotting_temperature *=  (moisture_anomaly_scaling/np.nanmax(np.abs(plotting_moisture[frame_to_plot])))
        plotting_moisture *=  (moisture_anomaly_scaling/np.nanmax(np.abs(plotting_moisture[frame_to_plot])))

    ##### Specify arguments for plotting the colors, contours, and quivers
    # This determines the magnitude of the maximum temperature anomaly, and sets the rounding appropriately
    temperature_order = -int(np.floor(np.log10(np.nanmax(np.abs(plotting_temperature[frame_to_plot]))))-1)
    contour_args = {
        'levels' :  np.delete(np.linspace(
                        -round_out(np.nanmax(np.abs(plotting_temperature[frame_to_plot])), temperature_order), 
                         round_out(np.nanmax(np.abs(plotting_temperature[frame_to_plot])), temperature_order), 
                         11
                    ), [5]),
        # 'levels' :  11,
        'norm'   :  mcolors.CenteredNorm(),
        'colors' :  'black'
    }

    if simulation_moisture:
        contourf_args = {
            'levels' :  np.linspace(
                            -round_out(np.nanmax(np.abs(plotting_moisture[frame_to_plot])), 'tenths'),
                             round_out(np.nanmax(np.abs(plotting_moisture[frame_to_plot])), 'tenths'),
                             21
                        ),
            'norm'   :  mcolors.CenteredNorm(),
            'cmap'   : Ahmed_cmap,
            'extend' : 'both'
        }

    quiver_scale = round_out(np.nanmax(np.abs(zonal_velocity[frame_to_plot])), 'tenths')
    quiver_args = {
        'color'       : 'k',
        'width'       : 0.0015,
        'angles'      : 'xy',
        'scale_units' : 'xy',
        'scale'       : quiver_scale
    }

    # Create a figure 
    fig = plt.figure(figsize=(16,6),dpi=500)
    
    # If the sim has mositure, add a colorbar
    if simulation_moisture:
        gs_main = gs.GridSpec(1, 2, width_ratios = [100,2], figure=fig)
        qbar_ax = fig.add_subplot(gs_main[-1])
        
    # Without moisture, just one axis is required
    else:
        gs_main = gs.GridSpec(1, 1, figure=fig)
        
    # Update the bounds and spacing of the gs.GridSpec
    gs_main.update(left=0.1, right=0.9, top=0.99, bottom=0.1, wspace=0.05)

    # Add an axis for the initial condition
    ax = fig.add_subplot(gs_main[0])

    # Label the plot
    if frame_to_plot == 0:
        ax.set_title(f"Initial condition", pad=10)
        output_time_string = "initial-condition"
        
    elif time_points[frame_to_plot] <= 120*3600:
        output_time_units = 'hr'
        output_time_string = f"{time_points[frame_to_plot]/(3600):0.1f}-{output_time_units}"
        ax.set_title(f"Time: {time_points[frame_to_plot]/3600:0.1f} {output_time_units}", pad=10)
    
    else:
        output_time_units = 'days'
        output_time_string = f"{time_points[frame_to_plot]/(24*3600):0.1f}-{output_time_units}"
        ax.set_title(f"Time: {time_points[frame_to_plot]/(24*3600):0.1f} {output_time_units}", pad=10)
        
    # Plot column temperature as a contour
    T_cont = ax.contour(
        zonal_gridpoints*grid_scaling,
        meridional_gridpoints*grid_scaling,
        plotting_temperature[frame_to_plot],
        **contour_args
    )
    ax.clabel(T_cont, T_cont.levels, inline=True, fontsize=10)

    # Plot column moisture as colors
    if simulation_moisture:
        q_cont = plt.contourf(
            zonal_gridpoints*grid_scaling,
            meridional_gridpoints*grid_scaling,
            plotting_moisture[frame_to_plot],
            **contourf_args
        )
        
        cbar = fig.colorbar(
            q_cont, 
            cax=qbar_ax
        )
        cbar.set_ticks(contourf_args['levels'][::4])
        cbar.set_label(moisture_units, rotation=0, labelpad=15)

    # Plot zonal and meridional velocity using vectors
    quiv = ax.quiver(
        zonal_gridpoints[::zonal_quiver_plot_spacing]*grid_scaling,
        meridional_gridpoints[::meridional_quiver_plot_spacing]*grid_scaling,
        -zonal_velocity[frame_to_plot][::meridional_quiver_plot_spacing,::zonal_quiver_plot_spacing],
        -meridional_velocity[frame_to_plot][::meridional_quiver_plot_spacing,::zonal_quiver_plot_spacing],
        **quiver_args
    )

    key_length = 0.05*(xlims[1]-xlims[0])*METERS_PER_DEGREE*grid_scaling
    key_label = f'{quiver_scale:0.2f} m/s'
    ax.quiverkey(
        quiv,          
        # 0.79, 1.02,          
        1.0, 1.05,          
        # U=2,        
        U=key_length,
        label=key_label,
        # label=f'{quiver_scale:0.2f} m/s',          
        # coordinates='figure', 
        labelpos='E',          
        color='black', 
        labelcolor='black'
    )

    # Add a dashed line signifying the equator
    ax.axhline(y=0, ls='--', alpha=0.5, color='gray', lw=0.5)

    if fringe_region:
        ax.axhline(y=fringe_region_latitude*METERS_PER_DEGREE*grid_scaling, color='k', ls=':')
        ax.axhline(y=-fringe_region_latitude*METERS_PER_DEGREE*grid_scaling, color='k', ls=':')
        
        ax.axhline(y=(fringe_region_latitude-fringe_region_width)*METERS_PER_DEGREE*grid_scaling, color='k', ls=':')
        ax.axhline(y=-(fringe_region_latitude-fringe_region_width)*METERS_PER_DEGREE*grid_scaling, color='k', ls=':')
    
    # Set tick labels in longitude/latitude coordinates
    longitude_ticks = np.arange(-180+60, 180+60, 60)
    longitude_labels = mjo.tick_labeller(longitude_ticks, direction='lon')
    ax.set_xticks(longitude_ticks*METERS_PER_DEGREE*grid_scaling, labels=longitude_labels)

    latitude_ticks = np.arange(-90, 90+20, 20)
    # latitude_ticks = np.arange(-10, 10+5, 5)
    latitude_labels = mjo.tick_labeller(latitude_ticks, direction='lat')
    ax.set_yticks(latitude_ticks*METERS_PER_DEGREE*grid_scaling, labels=latitude_labels)

    # Set the plot's meridional limits and aspect ratio 
    ax.set_xlim(xlims[0]*METERS_PER_DEGREE*grid_scaling,xlims[1]*METERS_PER_DEGREE*grid_scaling)
    ax.set_ylim(ylims[0]*METERS_PER_DEGREE*grid_scaling,ylims[1]*METERS_PER_DEGREE*grid_scaling)
    ax.set_aspect('auto')

    # Save the plot if specified
    if save_plot:
        print(
            f"Plot saved as: " 
          + f"{specified_output_file_directory}/{specified_initial_condition_name}_{output_time_string}" 
          + f"_horizontal_structure_{time.strftime('%Y%m%d-%H%M')}.png"
        )
        plt.savefig(
            f"{specified_output_file_directory}/{specified_initial_condition_name}_{output_time_string}" 
            + f"_horizontal_structure_{time.strftime('%Y%m%d-%H%M')}.png", 
            bbox_inches='tight'
        )
    else:
        plt.show()

## Animate solutions

In [22]:
def animate_horizontal_structure(
    zonal_gridpoints,
    meridional_gridpoints,
    time_points,
    zonal_velocity, 
    meridional_velocity, 
    column_temperature, 
    column_moisture, 
    specified_output_file_directory,
    specified_initial_condition_name,
    physical_parameters,
    simulation_parameters,
    plotting_parameters,
    starting_frame = 0,
    n_frames = 50,
    frame_interval = 50,
    normalized_over_time=True
):

    import time
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.animation import FuncAnimation
    plt.rcParams.update({'font.size':22})
    import matplotlib.ticker as mticker
    import matplotlib.colors as mcolors
    import matplotlib.gridspec as gs

    from tqdm import tqdm
    
    # Auxiliary Functions
    import sys
    sys.path.insert(0, '/home/disk/eos7/sressel/research/thesis-work/python/auxiliary_functions/')
    import ipynb.fs.full.mjo_mean_state_diagnostics as mjo
    from ipynb.fs.full.rounding_functions import round_out, round_to_multiple
    from ipynb.fs.full.modified_colormap import Ahmed21_colormap
    Ahmed_cmap = Ahmed21_colormap()
    from ipynb.fs.full.normalize_data import normalize_data
    
    [
        SPECIFIC_HEAT, 
        LATENT_HEAT,
        WATER_DENSITY,
        COLUMN_AVERAGE_MASS,
        EARTH_RADIUS,
        METERS_PER_DEGREE,
        SECONDS_PER_DAY,
    ] = physical_parameters

    [
        simulation_moisture,
        fringe_region
    ] = simulation_parameters
    
    [
        xlims, ylims,
        zonal_quiver_plot_spacing, meridional_quiver_plot_spacing,
        save_plot, plotting_units,
        moisture_anomaly_scaling,
        grid_scaling
    ] = plotting_parameters
    
    
    # Set plotting options
    plt.style.use('default')
    plt.rcParams.update({'font.size':22})
    modified_cmap = mjo.modified_colormap('bwr', 'white', 0.15, 0.05)

    # Specify whether to plot the values in simulated units or converted units
    if plotting_units == 'natural':
        plotting_temperature = column_temperature
        temperature_units = r"$\frac{K \, kg}{m^2}$"
        plotting_moisture = column_moisture
        moisture_units = r"$\frac{K \, kg}{m^2}$"

    elif plotting_units == 'converted':
        # Convert the units of temperature and moisture
        average_temperature = np.copy(column_temperature)/COLUMN_AVERAGE_MASS
        equivalent_moisture = np.copy(column_moisture)/LATENT_HEAT*SPECIFIC_HEAT*1000/WATER_DENSITY

        plotting_temperature = average_temperature
        temperature_units = r"K"
        plotting_moisture = equivalent_moisture
        moisture_units = r"mm"

    field_scaling_factor = (moisture_anomaly_scaling/np.nanmax(np.abs(plotting_moisture), axis=(1,2)))
    if (normalized_over_time == True) and (plotting_units == 'converted'):
        # print(f"Fields scaled! Maximum moisture anomaly set to {moisture_anomaly_scaling:0.2f} mm")
        zonal_velocity *=  field_scaling_factor[:, None, None]
        meridional_velocity *=  field_scaling_factor[:, None, None]
        plotting_temperature *=  field_scaling_factor[:, None, None]
        plotting_moisture *=  field_scaling_factor[:, None, None]
    
    # Specify arguments for plotting the colors, contours, and quivers
    contour_args = {
        # 'levels' :  np.delete(np.linspace(
        #                 -round_out(np.max(np.abs(plotting_temperature)), 'tenths'), 
        #                  round_out(np.max(np.abs(plotting_temperature)), 'tenths'), 
        #                  11
        #             ), [5]),
        'norm'   :  mcolors.CenteredNorm(),
        'colors' :  'black'
    }

    if simulation_moisture:
        contourf_args = {
            'levels' :  np.linspace(
                            -round_out(np.max(np.abs(plotting_moisture)), 'tenths'),
                             round_out(np.max(np.abs(plotting_moisture)), 'tenths'),
                             21
                        ),
            'norm'   :  mcolors.CenteredNorm(),
            'cmap'   : Ahmed_cmap
        }

    # quiver_scale = round_out(np.max(np.abs(zonal_velocity)), 'tenths')
    quiver_args = {
        'color'       : 'k',
        'width'       : 0.0025,
        'angles'      : 'xy',
        'scale_units' : 'xy',
        # 'scale'       : quiver_scale
    }

    # Animation parameters
    # starting_frame = 0
    
    # if (initial_wave == 'Kelvin') or (initial_wave == 'Rossby'): 
    #     # frame_interval = int((nt - starting_frame)/n_frames)
    frame_interval = int((np.shape(plotting_moisture)[0] - starting_frame)/n_frames)

    end_frame = starting_frame + n_frames*frame_interval
    frames = np.arange(starting_frame, end_frame, frame_interval)

    # Create a figure 
    fig = plt.figure(figsize=(16,6),dpi=300, layout='constrained')

    # If the sim has mositure, add a colorbar
    if simulation_moisture:
        gs_main = gs.GridSpec(1, 2, width_ratios = [100,2], figure=fig)
        qbar_ax = fig.add_subplot(gs_main[-1])

    # Without moisture, just one axis is required
    else:
        gs_main = gs.GridSpec(1, 1, figure=fig)

    # Update the bounds and spacing of the GridSpec
    gs_main.update(left=0.1, right=0.9, top=0.99, bottom=0.1, wspace=0.05)

    # Add an axis for the initial condition
    ax = fig.add_subplot(gs_main[0])

    # Label the plot
    ax.set_title(f"Time: {time_points[starting_frame]/SECONDS_PER_DAY:0.1f} days")

    # Plot column temperature as a contour
    T_cont = ax.contour(
        zonal_gridpoints*grid_scaling,
        meridional_gridpoints*grid_scaling,
        plotting_temperature[starting_frame],
        levels=np.delete(np.linspace(
                        -round_out(np.max(np.abs(plotting_temperature[starting_frame])), 'tenths'), 
                         round_out(np.max(np.abs(plotting_temperature[starting_frame])), 'tenths'), 
                         11
                    ), [5]),
        **contour_args
    )
    ax.clabel(T_cont, T_cont.levels, inline=True, fontsize=10)

    # Plot column moisture as colors
    if simulation_moisture:
        q_cont = plt.contourf(
            zonal_gridpoints*grid_scaling,
            meridional_gridpoints*grid_scaling,
            plotting_moisture[starting_frame],
            **contourf_args
        )

        cbar = fig.colorbar(
            q_cont, 
            cax=qbar_ax
        )
        cbar.set_ticks(contourf_args['levels'][::4])
        cbar.set_label(moisture_units, rotation=0, labelpad=15)

    # Plot zonal and meridional velocity using vectors
    quiv = ax.quiver(
        zonal_gridpoints[::zonal_quiver_plot_spacing]*grid_scaling,
        meridional_gridpoints[::meridional_quiver_plot_spacing]*grid_scaling,
        -zonal_velocity[starting_frame][::meridional_quiver_plot_spacing,::zonal_quiver_plot_spacing],
        -meridional_velocity[starting_frame][::meridional_quiver_plot_spacing,::zonal_quiver_plot_spacing],
        scale= round_out(np.max(np.abs(zonal_velocity[starting_frame])), 'tenths'),
        # scale = 200,
        **quiver_args
    )
    quiver_label_text = f"{round_out(np.max(np.abs(zonal_velocity[starting_frame])), 'tenths'):0.2f} m/s"

    # Create a temporary text object to calculate its width
    temp_text = ax.text(0., 0., quiver_label_text)

    # Calculate the width of the label text
    bbox = ax.transAxes.inverted().transform_bbox(
        temp_text.get_window_extent(
            renderer=plt.gcf().canvas.get_renderer()
        )
    )
    # renderer=plt.gcf().canvas.get_renderer()
    label_width = bbox.x1  - bbox.x0

    x_coordinate = 0.99
    adjusted_x = x_coordinate - label_width
    temp_text.set_visible(False)  # Make the text object invisible
    # ax.quiverkey(
    #     quiv,          
    #     adjusted_x, 1.035, 
    #     U=quiver_scale,          
    #     label=quiver_label_text,          
    #     coordinates='axes', labelpos='E',          
    #     color='black', labelcolor='black'
    # )

    key_length = 0.05*(xlims[1]-xlims[0])*METERS_PER_DEGREE*grid_scaling
    # key_label = f'200 m/s'
    key_label = f"{round_out(np.max(np.abs(zonal_velocity[starting_frame])), 'tenths'):0.2f} m/s"
    ax.quiverkey(
        quiv,          
        # 0.79, 1.02,          
        1.0, 1.05,          
        # U=2,        
        U=key_length,
        label=key_label,
        # label=f'{quiver_scale:0.2f} m/s',          
        # coordinates='figure', 
        labelpos='E',          
        color='black', 
        labelcolor='black'
    )

    # Add a dashed line signifying the equator
    ax.axhline(y=0, ls='--', alpha=0.5, color='gray', lw=0.5)

    # Set tick labels in longitude/latitude coordinates
    longitude_ticks = np.arange(-180+60, 180+60, 60)
    longitude_labels = mjo.tick_labeller(longitude_ticks, direction='lon')
    ax.set_xticks(longitude_ticks*METERS_PER_DEGREE*grid_scaling, labels=longitude_labels)

    latitude_ticks = np.arange(-50, 50+20, 20)
    # latitude_ticks = np.arange(-10, 10+5, 5)
    latitude_labels = mjo.tick_labeller(latitude_ticks, direction='lat')
    ax.set_yticks(latitude_ticks*METERS_PER_DEGREE*grid_scaling, labels=latitude_labels)

    # Set the plot's meridional limits and aspect ratio 
    ax.set_xlim(xlims[0]*METERS_PER_DEGREE*grid_scaling, xlims[1]*METERS_PER_DEGREE*grid_scaling)
    ax.set_ylim(ylims[0]*METERS_PER_DEGREE*grid_scaling, ylims[1]*METERS_PER_DEGREE*grid_scaling)

    # Set plot aspect
    ax.set_aspect('auto')

    def update(plotting_index):

        # Clear the plot each frame
        ax.clear()

        # Set the title to be the current time of the frame
        ax.set_title(f"Time: {time_points[plotting_index]/SECONDS_PER_DAY:0.1f} days")

        # Plot filled temperature contours
        T_cont = ax.contour(
            zonal_gridpoints*grid_scaling,
            meridional_gridpoints*grid_scaling,
            plotting_temperature[plotting_index],
            levels=np.delete(np.linspace(
                        -round_out(np.max(np.abs(plotting_temperature[plotting_index])), 'tenths'), 
                         round_out(np.max(np.abs(plotting_temperature[plotting_index])), 'tenths'), 
                         11
                    ), [5]),
            **contour_args
        )
        ax.clabel(T_cont, T_cont.levels, inline=True, fontsize=10)

        # Plot moisture contours
        if simulation_moisture:
            q_cont = plt.contourf(
                zonal_gridpoints*grid_scaling,
                meridional_gridpoints*grid_scaling,
                plotting_moisture[plotting_index],
                **contourf_args
            )

            cbar = fig.colorbar(
                q_cont, 
                cax=qbar_ax
            )
            cbar.set_ticks(contourf_args['levels'][::4])
            cbar.set_label(moisture_units, rotation=0, labelpad=15)


        # Plot wind vectors
        quiv = ax.quiver(
            zonal_gridpoints[::zonal_quiver_plot_spacing]*grid_scaling,
            meridional_gridpoints[::meridional_quiver_plot_spacing]*grid_scaling,
            -zonal_velocity[plotting_index,::meridional_quiver_plot_spacing,::zonal_quiver_plot_spacing],
            -meridional_velocity[plotting_index,::meridional_quiver_plot_spacing,::zonal_quiver_plot_spacing],
            scale = round_out(np.max(np.abs(zonal_velocity[plotting_index])), 'tenths'),
            # scale = 200,
            **quiver_args
        )

        # ax.quiverkey(
        #     quiv,          
        #     adjusted_x, 1.035,         
        #     U=quiver_scale,          
        #     label=quiver_label_text,          
        #     coordinates='axes', labelpos='E',          
        #     color='black', labelcolor='black'
        # )
        
        key_length = 0.05*(xlims[1]-xlims[0])*METERS_PER_DEGREE*grid_scaling
        key_label = f"{round_out(np.max(np.abs(zonal_velocity[plotting_index])), 'tenths'):0.2f} m/s"
        ax.quiverkey(
            quiv,          
            # 0.79, 1.02,          
            1.0, 1.05,          
            # U=2,        
            U=key_length,
            label=key_label,
            # label=f'{quiver_scale:0.2f} m/s',          
            # coordinates='figure', 
            labelpos='E',          
            color='black', 
            labelcolor='black'
        )

        # Set the x and y ticks to be longitude/latitude respectively
        ax.set_xticks(longitude_ticks*METERS_PER_DEGREE*grid_scaling, labels=longitude_labels)
        ax.set_yticks(latitude_ticks*METERS_PER_DEGREE*grid_scaling, labels=latitude_labels)
        ax.set_xlim(xlims[0]*METERS_PER_DEGREE*grid_scaling, xlims[1]*METERS_PER_DEGREE*grid_scaling)
        ax.set_ylim(ylims[0]*METERS_PER_DEGREE*grid_scaling ,ylims[1]*METERS_PER_DEGREE*grid_scaling)
        

        if simulation_moisture:
            return T_cont, quiv, q_cont

        else:
            return T_cont, quiv,

    # Run the animation
    anim = FuncAnimation(fig, update, frames=tqdm(frames, ncols=100, position=0, leave=True), interval=300)

    anim.save(
        f"{specified_output_file_directory}/{specified_initial_condition_name}_animation_{time.strftime('%Y%m%d-%H%M')}.mp4", 
        dpi=300
    )